# 3-SAT with qubovert

We'll first solve a 3-SAT problem with the `PCBO` class, and then solve it using the `qubovert.sat` library.

**Contents**

1. <a href="#PCBO">Solving 3-SAT with `qubovert.PCBO`</a>
2. <a href="#sat">Solving 3-SAT with the `qubovert.sat` library</a>
3. <a href="#combo">Solving 3-SAT with both!</a>

If `qubovert` is not `pip` installed and you are using these notebooks from within the `notebook_examples` directory, then run this cell. Otherwise it is not needed.

In [1]:
import sys
sys.path.append("..")

Import qubovert!

In [2]:
import qubovert

<div id="PCBO" />
    
## Solving 3-SAT with `qubovert.PCBO`

Create a Polynomial Constrained Boolean Optimization (``PCBO``) object.

In [3]:
H = qubovert.PCBO()

Let's try to encode the optimization problem of finding a satisfying assignment of $C$, where

$$C = c_0 \land c_1 \land c_2$$

where each $c_i$ is an OR clause of three variables:

$$c_0 = x_0 \lor x_1 \lor x_3, \quad c_1 = x_1 \lor \lnot x_2 \lor \lnot x_3, \quad c_2 = \lnot x_0 \lor x_2 \lor \lnot x_3.$$

Let's add these clauses to the PCBO.

We begin by adding constraints to enforce the NOTs.

In [4]:
H.add_constraint_eq_NOT("x_0", "-x_0")
H.add_constraint_eq_NOT("x_2", "-x_2")
H.add_constraint_eq_NOT("x_3", "-x_3")
print(H.constraints)

{'eq': [{('x_0',): -1, (): 1, ('-x_0',): -1}, {('x_2',): -1, (): 1, ('-x_2',): -1}, {('x_3',): -1, (): 1, ('-x_3',): -1}]}


Notice that the constraints are automatically added to the objective function. The `'eq'` key of the constraints dictionary indicates that the quantity equals zero. Other possible keys are `'lt'`, `'le'`, `'gt'`, `'ge'`, and `'ne'`. See the docstrings for `PCBO.add_constraint_eq_zero`, `PCBO.add_constraint_lt_zero`, `PCBO.add_constraint_le_zero`, `PCBO.add_constraint_gt_zero`, `PCBO.add_constraint_ge_zero`, and `PCBO.add_constraint_ne_zero` for info.

Now let's add the clauses to the PCBO (reproduced here for reference).

$$c_0 = x_0 \lor x_1 \lor x_3, \quad c_1 = x_1 \lor \lnot x_2 \lor \lnot x_3, \quad c_2 = \lnot x_0 \lor x_2 \lor \lnot x_3.$$

In [5]:
H.add_constraint_OR("x_0", "x_1", "x_3")
H.add_constraint_OR("x_1", "-x_2", "-x_3")
H.add_constraint_OR("-x_0", "x_2", "-x_3")
print("\n\n".join(str(x) for x in H.constraints["eq"]))

{('x_0',): -1, (): 1, ('-x_0',): -1}

{('x_2',): -1, (): 1, ('-x_2',): -1}

{('x_3',): -1, (): 1, ('-x_3',): -1}

{('x_0',): -1, ('x_0', 'x_1'): 1, ('x_1',): -1, ('x_0', 'x_3'): 1, ('x_0', 'x_3', 'x_1'): -1, ('x_3', 'x_1'): 1, ('x_3',): -1, (): 1}

{('x_1',): -1, ('-x_2', 'x_1'): 1, ('-x_2',): -1, ('-x_3', 'x_1'): 1, ('-x_2', '-x_3', 'x_1'): -1, ('-x_2', '-x_3'): 1, ('-x_3',): -1, (): 1}

{('-x_0',): -1, ('-x_0', 'x_2'): 1, ('x_2',): -1, ('-x_0', '-x_3'): 1, ('-x_0', '-x_3', 'x_2'): -1, ('-x_3', 'x_2'): 1, ('-x_3',): -1, (): 1}


Here is the final PUBO formulation of the problem. Finding a satisfying assignment is equivalent to minimizing it.

In [6]:
H

{('x_0', '-x_0'): 2,
 ('x_0',): -2,
 (): 6,
 ('-x_0',): -2,
 ('-x_2', 'x_2'): 2,
 ('x_2',): -2,
 ('-x_2',): -2,
 ('-x_3', 'x_3'): 2,
 ('x_3',): -2,
 ('-x_3',): -3,
 ('x_0', 'x_1'): 1,
 ('x_1',): -2,
 ('x_0', 'x_3'): 1,
 ('x_0', 'x_3', 'x_1'): -1,
 ('x_3', 'x_1'): 1,
 ('-x_2', 'x_1'): 1,
 ('-x_3', 'x_1'): 1,
 ('-x_2', '-x_3', 'x_1'): -1,
 ('-x_2', '-x_3'): 1,
 ('-x_0', 'x_2'): 1,
 ('-x_0', '-x_3'): 1,
 ('-x_0', '-x_3', 'x_2'): -1,
 ('-x_3', 'x_2'): 1}

Here we see the full PUBO representing the PCBO. Let's solve it bruteforce to make sure everything is working well. `H.solve_bruteforce` will ensure that all the constraints are satisfied.

In [7]:
print("number of variables:", H.num_binary_variables)
H_solutions = H.solve_bruteforce(all_solutions=True)
print("number of solutions:", len(H_solutions))

number of variables: 7
number of solutions: 10


We see there are 10 possible solutions to this 3-SAT problem.

Now let's solve this problem with a generic QUBO solver. Notice that the degree of problem is more than two, making `H` not a natural Quadratic Unconstrained Boolean Optimization Problem (QUBO).

In [8]:
H.degree

3

We can convert it to a QUBO (note that there are some options for the reduction from PUBO to QUBO, see the `H.to_qubo` method for details). Ancilla bits will need to be added, and bit labels are mapped to integers.

In [9]:
Q = H.to_qubo()
print(Q)

{(0, 1): 2, (0,): -2, (1,): -2, (2, 3): 2, (3,): -2, (2,): -2, (4, 5): 2, (5,): -2, (4,): -3, (0, 6): 1, (6,): -2, (0, 5): 3, (7,): 6, (0, 7): -4, (5, 7): -4, (6, 7): -1, (5, 6): 1, (2, 6): 1, (4, 6): 1, (8,): 6, (2, 4): 3, (2, 8): -4, (4, 8): -4, (6, 8): -1, (1, 3): 3, (1, 4): 1, (9,): 6, (1, 9): -4, (3, 9): -4, (4, 9): -1, (3, 4): 1, (): 6}


For testing purposes, let's solve this with bruteforce. Notice how we remap the QUBO solution to the PCBO solution with `H.convert_solution(x)`.

In [10]:
print("number of variables:", Q.num_binary_variables)
Q_solutions = [H.convert_solution(x) for x in Q.solve_bruteforce(all_solutions=True)]
print("number of solutions:", len(Q_solutions))
print("Q solutions", "do" if Q_solutions == H_solutions else "do not", "match the H solutions")

number of variables: 10
number of solutions: 10
Q solutions do match the H solutions


Now let's solve the QUBO with D'Wave's simulated annealer.

In [11]:
#!pip install dwave-neal
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

Note that their software package takes in a specific form for QUBOs, namely, the keys of the dictionary must be two element tuples. This form can be accessed from `Q` with `Q.Q`.

In [12]:
qubo_sample = sampler.sample_qubo(Q.Q, num_reads=500)
print("objective function:", qubo_sample.first.energy + Q.offset, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution, "\n")

solution = H.convert_solution(qubo_solution)
print("pcbo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: 0.0 

qubo solution: {0: 0, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1} 

pcbo solution: {'x_0': 0, '-x_0': 1, '-x_2': 0, 'x_2': 1, '-x_3': 0, 'x_3': 1, 'x_1': 1}
objective function: 0 

The solution is valid


Notice that `H.is_solution_valid` checks if all the constraints are satisfied.

Now we'll solve an QUSO formulation of our problem.

In [13]:
L = H.to_quso()
print(L)

{(0, 1): 0.5, (1,): 0.5, (0,): 0.5, (2, 3): 0.5, (3,): 0.5, (2,): 0.5, (4, 5): 0.5, (5,): 0.5, (0, 6): 0.25, (6,): 0.5, (0, 5): 0.75, (7,): -0.75, (0, 7): -1.0, (5, 7): -1.0, (6, 7): -0.25, (5, 6): 0.25, (2, 6): 0.25, (4, 6): 0.25, (8,): -0.75, (2, 4): 0.75, (2, 8): -1.0, (4, 8): -1.0, (4,): 0.75, (6, 8): -0.25, (1, 3): 0.75, (1, 4): 0.25, (9,): -0.75, (1, 9): -1.0, (3, 9): -1.0, (4, 9): -0.25, (3, 4): 0.25, (): 6}


Similar to their QUBO solver, D'Wave's QUSO solver accepts a specific form for QUSO models, namely a linear term dictionary and a quadratic term dictionary. These can be accessed with `L.h` and `L.J`.

In [14]:
quso_sample = sampler.sample_quso(L.h, L.J, num_reads=500)
print("objective function:", quso_sample.first.energy + L.offset, "\n")

quso_solution = quso_sample.first.sample
print("quso solution:", quso_solution, "\n")

solution = H.convert_solution(quso_solution)
print("pcbo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: 0.0 

quso solution: {0: 1, 1: -1, 2: -1, 3: 1, 4: -1, 5: 1, 6: -1, 7: 1, 8: -1, 9: 1} 

pcbo solution: {'x_0': 0, '-x_0': 1, '-x_2': 1, 'x_2': 0, '-x_3': 1, 'x_3': 0, 'x_1': 1}
objective function: 0 

The solution is valid


We see that the solution is again valid.

<div id="sat" />

## Solving 3-SAT with the `qubovert.sat` library

Here we will use the ``qubovert.sat`` library to do a better job encoding the 3-SAT problem discussed above, and reproduced below.

Find a satisfying assignment of $C$, where

$$C = c_0 \land c_1 \land c_2$$

where each $c_i$ is an OR clause of three variables:

$$c_0 = x_0 \lor x_1 \lor x_3, \quad c_1 = x_1 \lor \lnot x_2 \lor \lnot x_3, \quad c_2 = \lnot x_0 \lor x_2 \lor \lnot x_3.$$

In [15]:
from qubovert.sat import AND, OR, NOT

c0 = OR('x_0', 'x_1', 'x_3')
c1 = OR('x_1', NOT('x_2'), NOT('x_3'))
c2 = OR(NOT('x_0'), 'x_2', NOT('x_3'))

C = AND(c0, c1, c2)
print(C)

{('x_0',): 1, ('x_0', 'x_3'): -2, ('x_0', 'x_1'): -1, ('x_1',): 1, ('x_3', 'x_1'): -1, ('x_0', 'x_3', 'x_1'): 1, ('x_3',): 1, ('x_0', 'x_3', 'x_2'): 1, ('x_3', 'x_1', 'x_2'): 1, ('x_3', 'x_2'): -1}


``C`` is a ``qubovert.PUBO`` object that is representing the boolean expression $C$. It is 1 for a satisfying solution, and 0 otherwise. Thus, we want to minimize ``P = -C``.

In [16]:
P = -C
print("Number of variables:", P.num_binary_variables)
print("degree:", P.degree)

Number of variables: 4
degree: 3


Let's do the same as before and solve ``P`` bruteforce.

In [17]:
solutions = P.solve_bruteforce(all_solutions=True)
print("number of valid solutions:", len(solutions))

number of valid solutions: 10


Notice that this matches the PCBO solutions from before, but there are many less variables in the PUBO this time.

Now let's solve the QUBO with D'Wave's simulated annealer.

In [18]:
Q = P.to_qubo()

print("Number of QUBO variables:", Q.num_binary_variables, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution)

qubo_sample = sampler.sample_qubo(Q.Q, num_reads=500)
print("qubo objective function:", qubo_sample.first.energy + Q.offset, "\n")

solution = P.convert_solution(qubo_solution)
print("pubo solution:", solution)
print("pubo objective function:", P.value(solution), "\n")

print("C objective function (1 if satisfying else 0):", C.value(solution), "\n")

Number of QUBO variables: 6 

qubo solution: {0: 0, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1}
qubo objective function: -1.0 

pubo solution: {'x_0': 0, 'x_3': 1, 'x_1': 0, 'x_2': 1}
pubo objective function: 0 

C objective function (1 if satisfying else 0): 0 



Notice that the QUBO formulation also has many less variables than before. 

Now let's solve the QUSO.

In [19]:
L = P.to_quso()

print("Number of QUSO variables:", L.num_binary_variables, "\n")

quso_solution = quso_sample.first.sample
print("quso solution:", quso_solution)

quso_sample = sampler.sample_quso(L.h, L.J, num_reads=500)
print("quso objective function:", quso_sample.first.energy + L.offset, "\n")

solution = P.convert_solution(quso_solution)
print("pubo solution:", solution)
print("pubo objective function:", P.value(solution), "\n")

print("C objective function (1 if satisfying else 0):", C.value(solution), "\n")

Number of QUSO variables: 6 

quso solution: {0: 1, 1: -1, 2: -1, 3: 1, 4: -1, 5: 1, 6: -1, 7: 1, 8: -1, 9: 1}
quso objective function: -1.0 

pubo solution: {'x_0': 0, 'x_3': 1, 'x_1': 1, 'x_2': 0}
pubo objective function: -1 

C objective function (1 if satisfying else 0): 1 



Thus we have solved the 3-SAT problem! Notice that the ``qubovert.sat`` library uses less variables to encode a SAT problem as does naively using the PCBO formulation.

If we wanted to, for example, minimize some other PUBO ``W`` subject to the constraint that ``C`` is satisfied, then we could simply add a penalty ``1 - C`` to ``W``. Ie the minimum of ``W + lam * (1 - C)`` will satisfy ``C == 1`` provided that ``lam`` is large enough.

<div id="combo" />

## Solving 3-SAT with both!

Here we show a different way to encode the 3-SAT problem. We reproduce the problem here again.

Find a satisfying assignment of $C$, where

$$C = c_0 \land c_1 \land c_2$$

where each $c_i$ is an OR clause of three variables:

$$c_0 = x_0 \lor x_1 \lor x_3, \quad c_1 = x_1 \lor \lnot x_2 \lor \lnot x_3, \quad c_2 = \lnot x_0 \lor x_2 \lor \lnot x_3.$$

In [20]:
from qubovert import boolean_var, PCBO
from qubovert.sat import NOT

x = [boolean_var("x%d" % i) for i in range(4)]

H = PCBO().add_constraint_eq_OR(
    'c0', x[0], x[1], x[3]
).add_constraint_eq_OR(
    'c1', x[1], NOT(x[2]), NOT(x[3])
).add_constraint_eq_OR(
    'c2', NOT(x[0]), x[2], NOT(x[3])
).add_constraint_AND(
    'c0', 'c1', 'c2'
)

print("degree:", H.degree)
print("num variables:", H.num_binary_variables)
print("num of solutions:", len(H.solve_bruteforce(all_solutions=True)))

degree: 4
num variables: 7
num of solutions: 10


`H` is a PCBO object, so we can do all the same things as we did in the previous sections. Ie convert to a QUBO or QUSO and solve.